### 8.1 python标准库的时间处理函数


#### 一. datetime模块
1. 当前时间 : `datetime.now`
2. datetime对象初始化 : `datetime(year,month,day,hour,minutes,seconds,miliseconds,timezone)`, 参数都是int型

In [8]:
from datetime import datetime 
print datetime.now()
print datetime(1991,7,31,11,20,2)

2018-06-19 11:25:24.463271
1991-07-31 11:20:02
